In [336]:
from collections import deque, OrderedDict, defaultdict
import time

import numpy as np

In [337]:
with open('data/gc_50_3', 'r') as input_data_file:
    input_data = input_data_file.read()

lines = input_data.split('\n')

first_line = lines[0].split()
node_count = int(first_line[0])
edge_count = int(first_line[1])

In [338]:
node_count

50

In [339]:
edge_count

350

In [340]:
test = defaultdict(list)

In [341]:
test['a']

[]

In [342]:
A = np.zeros([node_count,node_count])

In [343]:
edges = []
adjacent_nodes = defaultdict(list)
for i in range(1, edge_count + 1):
    line = lines[i]
    parts = line.split()
    edges.append((int(parts[0]), int(parts[1])))
    A[int(parts[0])][int(parts[1])] = 1
    A[int(parts[1])][int(parts[0])] = 1
    adjacent_nodes[int(parts[0])].append(int(parts[1]))
    adjacent_nodes[int(parts[1])].append(int(parts[0]))


In [344]:
A

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [346]:
def color_graph(graph, counter):
    
    while counter:
        stack = deque([[k for k, v in sorted(counter.items(), key=lambda item: item[1], reverse=True)][0]])
        node = stack.pop()
        counter.pop(node, None)

        color_set = list(set([graph[i]['color'] for i in graph[node]['adjacent']]))

        if len(color_set) == 1 and color_set[0] == -1:
            graph[node]['color'] = 0
        else:
            possible = set(list(range(0, max(color_set)+1)))
            diff = possible.difference(set(color_set))
            if diff:
                color = min(diff)
            else:
                color = max(possible)+1
            graph[node]['color'] = color

        for i in graph.keys():
            if node in graph[i]["pruning"]:
                graph[i]["pruning"].remove(node)

        for i in counter.keys():
            counter[i] = len(graph[i]["pruning"])
            
    return graph

In [347]:
def update_graph(graph, counter, node_1, node_2):

    if node_1 not in graph:
        graph[node_1] = {'adjacent': (node_2,), 'pruning': [node_2], 'color': -1, 'update_counter': 1}
        counter[node_1] = 1
    else:
        counter[node_1] += 1
        graph[node_1]['update_counter'] += 1
        graph[node_1]['adjacent'] += (node_2,)
        graph[node_1]['pruning'].append(node_2)

    return graph, counter


def get_graphs(edges):

    graph = OrderedDict()
    counter = {}
    for edge in edges:
        graph, counter = update_graph(graph, counter, edge[0], edge[1])
        graph, counter = update_graph(graph, counter, edge[1], edge[0])

    return graph, counter


def color_graph(graph, counter):

    while counter:
        stack = deque([[k for k, v in sorted(counter.items(), key=lambda item: item[1], reverse=True)][0]])
        node = stack.pop()
        counter.pop(node, None)

        color_set = list(set([graph[i]['color'] for i in graph[node]['adjacent']]))

        if len(color_set) == 1 and color_set[0] == -1:
            graph[node]['color'] = 0
        else:
            possible = set(list(range(0, max(color_set)+1)))
            diff = possible.difference(set(color_set))
            if diff:
                color = min(diff)
            else:
                color = max(possible)+1
            graph[node]['color'] = color

        for i in graph.keys():
            if node in graph[i]["pruning"]:
                graph[i]["pruning"].remove(node)

        for i in counter.keys():
            counter[i] = len(graph[i]["pruning"])

    return graph

In [352]:
solution = node_count*[0]
for node, value in graph.items():
    solution[node] = value['color']

In [351]:
for edge in edges:
    if solution[edge[0]] == solution[edge[1]]:
        print(edge)

or-tools

In [354]:
from ortools.sat.python import cp_model

In [383]:
def or_solver(node_count, edges):

    solved = False
    max_color = 1

    while not solved:

        max_color += 1

        model = cp_model.CpModel()
        variables = [model.NewIntVar(0, max_color-1, f'x{i}') for i in range(node_count)]
        for edge in edges:
            model.Add(variables[edge[0]] != variables[edge[1]])

        solver = cp_model.CpSolver()
        solver.parameters.max_time_in_seconds = 30.0
        status = solver.Solve(model)

        if status == cp_model.FEASIBLE:
            print("solution feasible")
            solved = True
            solution = [solver.Value(var) for var in variables]

    return max_color, solution

In [384]:
max_color, solution = or_solver(node_count, edges)

solution feasible
